# Assignment 6


In [4]:
#Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 101)

###Data Description

Column | Description
:---|:---
`id` | Record index
`timestamp` | Datetime (YYYY:MM:DD HH:MM:SS) when data was collected
`country` | Current country of employment
`employment_status` | Whether a candidate is Full time, Part time, Independent or freelancer or company owner
`job_title` | Current job title of the candidate
`job_years` | Total job experience (in Years)
`is_manager` | Whether the candidate holds a managerial position or not (Yes or No)
`hours_per_week` | No. of hours per day committed to the current job
`telecommute_days_per_week` | No. of telecommuting days per week (working from home)
`education` | The highest degree in education the candidate has received
`is_education_computer_related` | Is the education related to the field of computer science (Yes or No)
`certifications` | Does the candidate have any relevant certifications (Yes or No)
`salary` | Monthly Salary (in US $$)

In [ ]:
# Dataset is already loaded below
data = pd.read_csv('path/to/employee.csv')

In [11]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/employee.csv')  # Adjust path as needed
print(data.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   id            timestamp        country employment_status  job_title  \
0   1  12/11/2018 10:52:26       Slovenia         Full time  Developer   
1   2    1/5/2017 16:57:50  United States         Full time        DBA   
2   3   12/18/2017 8:13:15         Sweden         Full time        DBA   
3   4   12/27/2018 4:56:52  United States         Full time        DBA   
4   5  12/11/2018 14:07:58  United States         Full time  Developer   

   job_years is_manager  hours_per_week  telecommute_days_per_week  \
0    4.78393        Yes            40.0                        0.0   
1    5.00000         No            40.0                        5.0   
2    1.00000         No            40.0                        0.0   
3    1.00000         No            40.0                        2.0   
4    3.00000         No            40.0                        2.0   

     

In [12]:
# Dimensions of training data
data.shape

(4277, 13)

In [13]:
# Print first few rows of data
data.head()

,id,timestamp,country,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,1,12/11/2018 10:52:26,Slovenia,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,2,1/5/2017 16:57:50,United States,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,3,12/18/2017 8:13:15,Sweden,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,4,12/27/2018 4:56:52,United States,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,5,12/11/2018 14:07:58,United States,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000


In [14]:
# drop id, timestamp and country columns
data = data.drop(columns=['id', 'timestamp','country'])

In [15]:
# Explore columns
data.columns

Index(['employment_status', 'job_title', 'job_years', 'is_manager',
       'hours_per_week', 'telecommute_days_per_week', 'education',
       'is_education_computer_related', 'certifications', 'salary'],
      dtype='object')

In [16]:
# replace NANs in hours_per_week with median value of the column
data.loc[data['hours_per_week'].isna(), 'hours_per_week'] = data['hours_per_week'].median()
data.loc[data['telecommute_days_per_week'].isna(), 'telecommute_days_per_week'] = data['telecommute_days_per_week'].median()

In [17]:
#Handling null values in categorical columns
data = data.dropna()

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4261 entries, 0 to 4276
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   employment_status              4261 non-null   object 
 1   job_title                      4261 non-null   object 
 2   job_years                      4261 non-null   float64
 3   is_manager                     4261 non-null   object 
 4   hours_per_week                 4261 non-null   float64
 5   telecommute_days_per_week      4261 non-null   float64
 6   education                      4261 non-null   object 
 7   is_education_computer_related  4261 non-null   object 
 8   certifications                 4261 non-null   object 
 9   salary                         4261 non-null   float64
dtypes: float64(4), object(6)
memory usage: 366.2+ KB


###Data Visualization :

## Feature Encoding and Normalization

Before training the model, we should perform one-hot encoding for all categorical/discrete variables, normalize continuous variables and then combine all data to form the training set.

In [19]:
# create another copy of dataset and append encoded features to it
data_train = data.copy()
data_train.head()

,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000


In [20]:
# select categorical features
cat_cols = [c for c in data_train.columns if data_train[c].dtype == 'object'
            and c not in ['is_manager', 'certifications']]
cat_data = data_train[cat_cols]
cat_cols

['employment_status',
 'job_title',
 'education',
 'is_education_computer_related']

In [21]:
#Encoding binary variables
binary_cols = ['is_manager', 'certifications']
for c in binary_cols:
    data_train[c] = data_train[c].replace(to_replace=['Yes'], value=1)
    data_train[c] = data_train[c].replace(to_replace=['No'], value=0)

In [22]:
final_data = pd.get_dummies(data_train, columns=cat_cols, drop_first= True,dtype=int)
final_data.shape

(4261, 25)

In [23]:
final_data.columns

Index(['job_years', 'is_manager', 'hours_per_week',
       'telecommute_days_per_week', 'certifications', 'salary',
       'employment_status_Independent or freelancer or company owner',
       'employment_status_Part time', 'job_title_Analytics consultant',
       'job_title_Architect', 'job_title_DBA', 'job_title_Data Scientist',
       'job_title_Developer', 'job_title_Engineer', 'job_title_Manager',
       'job_title_Other', 'job_title_Principal database engineer',
       'job_title_Sales', 'job_title_Sr Consultant ',
       'education_Bachelors (4 years)', 'education_Doctorate/PhD',
       'education_Masters', 'education_None (no degree completed)',
       'is_education_computer_related_Unknown',
       'is_education_computer_related_Yes'],
      dtype='object')

In [24]:
final_data

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,salary,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
0,4.783930,1,40.0,0.0,0,7187.743094,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
1,5.000000,0,40.0,5.0,0,10000.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,1.000000,0,40.0,0.0,1,7000.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,1.000000,0,40.0,2.0,0,8333.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,3.000000,0,40.0,2.0,1,7137.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,2.000000,1,40.0,0.0,0,4917.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4273,4.140793,1,40.0,0.0,0,7033.845701,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4274,5.000000,0,40.0,0.0,0,7166.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4275,4.000000,1,40.0,2.0,0,9583.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1


## Train Test Split

In [25]:
y = final_data['salary']
X = final_data.drop(columns=['salary'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Training Set Dimensions:", X_train.shape)
print("Validation Set Dimensions:", X_test.shape)

Training Set Dimensions: (3408, 24)
Validation Set Dimensions: (853, 24)


## Pre-processing data

### Standardization (Z-score normalization):

$$ x_{\text{std}} = \frac{x - \mu}{\sigma} $$

- **Purpose:** Standardization transforms the data to have a mean of 0 and a standard deviation of 1.
- **Properties:**
  - Centers the data around 0.
  - Rescales the data to have unit variance.
  - Does not bound the data within a specific range.
  - Preserves the shape of the distribution.
- **Use Cases:**
  - Algorithms that assume zero-centered data or require features to have a similar scale (e.g., gradient descent-based algorithms, support vector machines).
  - When the distribution of the features is Gaussian-like.

### Normalization (Min-Max scaling):

 $$ x_{\text{norm}} = \frac{x - \min(x)}{\max(x) - \min(x)} $$

- **Purpose:** Normalization scales the data to a fixed range, typically [0, 1] or [-1, 1].
- **Properties:**
  - Scales the data to a specified range.
  - Shifts the data to start at 0.
  - Does not affect the shape of the distribution.
  - Preserves the relative relationships between data points.
- **Use Cases:**
  - Neural networks, especially those with activation functions sensitive to input magnitudes (e.g., sigmoid or tanh functions).
  - When the distribution of the features is unknown or non-Gaussian.

**Choosing Between Standardization and Normalization:**
- Use standardization when the distribution of your features is approximately Gaussian-like and you want to center the data and rescale it to have unit variance.
- Use normalization when the scale of your features is important, or when you need to bound the features within a specific range.
- It's often beneficial to try both preprocessing techniques and evaluate their effects on model performance to determine which one works best for your specific dataset and model.


In [26]:
# select numerical features
num_cols = ['job_years','hours_per_week','telecommute_days_per_week']
num_cols


['job_years', 'hours_per_week', 'telecommute_days_per_week']

In [27]:
# Apply standard scaling on numeric data
scaler = StandardScaler()
scaler.fit(X_train[num_cols])
X_train[num_cols] = scaler.transform(X_train[num_cols])

In [28]:
X_train

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
3576,-0.013444,0,0.015472,-0.624597,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
2792,0.181062,1,-0.214318,-0.624597,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1536,0.072018,1,-0.438225,-0.012747,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
1457,-1.249535,0,-0.490580,-0.012747,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
2538,0.643827,0,-0.305737,-0.012747,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,-1.249535,0,2.339409,-0.624597,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3574,-0.111603,0,0.120298,-0.012747,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
919,-0.784164,0,-0.490580,0.599103,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3495,0.260070,0,-0.490580,-0.624597,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [29]:
#Fitting a Linear Regression Model
reg=LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [31]:
reg.coef_

array([ 1.83991353e+02,  5.28243401e+01,  2.27164445e+02,  2.37665939e+02,
        4.55533729e+01, -5.56538063e+01,  1.21747942e+02, -2.26100177e+03,
        1.18437172e+03,  8.93383762e+02,  8.94943194e+02,  5.67395845e+02,
        1.03886362e+03,  9.39150058e+02, -2.12209832e+01,  1.02424211e+03,
        7.68780424e+02, -1.25055521e-12,  3.49099395e+02, -2.30978601e+02,
        1.93286901e+02,  1.60321435e+02, -2.46397621e+02, -1.55599109e+02])

In [32]:
reg.intercept_

6114.741340058091

Just to recall

$\hat{y} = \alpha + \beta_1 * X_1 + \beta_2 * X_2 +...$

Our Final model is given by -

$\hat{y} = 6145.79 + 1.887 * X_1 + 7.22 * X_2 +...$


In [33]:
# Normalized MSE (Dividing by mean)
mean_squared_error(y_train,reg.predict(X_train))/np.mean(y_train)

202.28906658607178

In [34]:
# Predict on the test data
y_pred = reg.predict(X_test)

In [35]:
#Evaluate the model on test data
mse = mean_squared_error(y_pred, y_test)/np.mean(y_test)
print("Mean Squared Error:", mse)


Mean Squared Error: 15417.207043990962


*A lower MSE indicates that the model's predictions are closer to the actual values on average, while a higher MSE suggests larger errors between predictions and actual values.
❗Pre-processing on Test data not done.

In [ ]:
#Q1. Preprocess Test data and get predictions

In [36]:
scaler.fit(X_test[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

In [38]:
X_test

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
2243,1.760435,0,-0.479313,-0.633548,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0
482,-0.176610,0,-0.003548,-0.633548,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
3855,1.262131,0,0.696493,-0.633548,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
770,0.265521,1,-0.479313,1.708452,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1148,-0.248769,0,0.422244,2.293953,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2562,0.086545,0,0.175692,-0.633548,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2225,-0.146924,0,-0.208580,0.537452,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0
2621,2.757044,0,-0.479313,-0.633548,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
2027,0.065581,0,-0.369893,0.537452,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0


In [39]:
y_hat = reg.predict(X_test)

In [40]:
mae_test= mean_absolute_error(y_test,y_hat)/np.mean(y_hat)
print("Test Mean Abosulte Error: ",mae_test)

Test Mean Abosulte Error:  0.13206683094565905


In [41]:
mse_test = mean_squared_error(y_test,y_hat)/np.mean(y_hat)
print("Test Mean Squared Error: ",mse_test)

Test Mean Squared Error:  239.70195722381132
